In [67]:
# importing required libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error as msle

In [68]:
# reading the dataset
train_data = pd.read_csv('dataset/train.csv')
product_data = pd.read_csv('dataset/product_data.csv')
store_data = pd.read_csv('dataset/store_data.csv')





In [69]:
### Replace the null values of parking quantity with a value that is directly proportional to the sales area

grouped = store_data.groupby('SEG_VALUE_NAME').agg({
    'PARKING_SPACE_QTY': 'sum',
    'SALES_AREA_SIZE_NUM': 'sum'
})
print(grouped)
grouped['park_ratio']=grouped['PARKING_SPACE_QTY'] / grouped['SALES_AREA_SIZE_NUM']
print(grouped)
store_data['park_ratio']=store_data['SEG_VALUE_NAME'].map(grouped['park_ratio'])

store_data['PARKING_SPACE_QTY'] = store_data.apply(
    lambda row: row['park_ratio'] * row['SALES_AREA_SIZE_NUM'] if pd.isnull(row['PARKING_SPACE_QTY']) else row['PARKING_SPACE_QTY'], axis=1
)
store_data.drop(columns=['park_ratio'], inplace=True)


                PARKING_SPACE_QTY  SALES_AREA_SIZE_NUM
SEG_VALUE_NAME                                        
MAINSTREAM                 5920.0              2153267
UPSCALE                    1859.0               833790
VALUE                      4412.0               735421
                PARKING_SPACE_QTY  SALES_AREA_SIZE_NUM  park_ratio
SEG_VALUE_NAME                                                    
MAINSTREAM                 5920.0              2153267    0.002749
UPSCALE                    1859.0               833790    0.002230
VALUE                      4412.0               735421    0.005999


In [70]:
### I'm converting the product_size column into INT and and adding 2 columns "NUmber of alternatives" and number of "alternatives of lower size". 

import re
product_data['PRODUCT_SIZE'] = product_data['PRODUCT_SIZE'].apply(lambda x: float(re.findall(r'\d*\.?\d+', x)[0]))

product_data['num_alternatives'] = product_data.groupby('SUB_CATEGORY')['UPC'].transform('count') - 1
def count_lower_size(row, product_data):
    group = product_data[(product_data['SUB_CATEGORY'] == row['SUB_CATEGORY']) &
               (product_data['PRODUCT_SIZE'] < row['PRODUCT_SIZE'])]
    return len(group)

product_data['num_lower_size_alternatives'] = product_data.apply(count_lower_size, axis=1, product_data=product_data)

product_data.to_csv('update_product_data.csv')


In [71]:


merged_data = train_data.merge(product_data, how= 'left', on= 'UPC')
merged_data = merged_data.merge(store_data,how='left', left_on = 'STORE_NUM', right_on='STORE_ID')

In [72]:
# convert to datetime
merged_data.WEEK_END_DATE = pd.to_datetime(merged_data.WEEK_END_DATE)


# create an array of unique week dates
week = merged_data.WEEK_END_DATE.unique()

merged_data['year'] = merged_data['WEEK_END_DATE'].dt.year
merged_data['week_number'] = merged_data['WEEK_END_DATE'].dt.isocalendar().week

merged_data = merged_data.sort_values(by=['UPC', 'STORE_NUM', 'year', 'week_number'])
merged_data['units_sold_prev_year'] = merged_data.groupby(['UPC', 'STORE_NUM', 'week_number'])['UNITS'].shift(1)

/var/folders/k0/mx_3vf5n6wdg8wkvxkg7znt00000gp/T/ipykernel_95868/3316337609.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_data.WEEK_END_DATE = pd.to_datetime(merged_data.WEEK_END_DATE)


In [73]:

# Convert all the categorical data using one-hot encoding



In [74]:
from datetime import timedelta
def validation_df(data, week, no_of_months, no_of_validation):
    
    model_set = []
    set_n = 1
    for w in range(len(week)-1,0,-1):
        x_data = {}

        x_data['train_start'] = week[w-3-4*no_of_months]
        x_data['train_end'] = week[w-4]
        x_data['validate_week'] = week[w-2]
        x_data['test_week'] = week[w]
        x_data['no_days_train'] = x_data['train_end'] - x_data['train_start']
        x_data['set_no'] = 'set'+str(set_n)
        set_n +=1
        model_set.append(x_data)
        if(len(model_set) == no_of_validation):
            break
        
        
    datapoints = []

    for s in model_set :
        x = {}
        train_set = data[(data.WEEK_END_DATE >= s['train_start']) & (data.WEEK_END_DATE <= s['train_end'])]
        x['train_shape'] = train_set.shape[0]
        x['validation_shape']  = data[data.WEEK_END_DATE == s['validate_week']].shape[0]
        x['test_shape'] = data[data.WEEK_END_DATE == s['test_week']].shape[0]
        x.update(s)
        datapoints.append(x)

    df = pd.DataFrame.from_dict(datapoints)
    df['no_days_train'] = df['no_days_train'] + timedelta(days=7)
    return df    

In [75]:


# function to calculate the root mean squared log error
def get_msle(true, predicted) :
    return np.sqrt(msle(true, predicted))

# function to return the columns on which the model is trained
def get_colums(data):
    print('\n####### The model is trained on Following Columns: ###########\n')
    print(data.columns)
    print('===============================================================')


# function to train the model 
# it will calculate and return the RMSLE on train and validation set    
def my_model(train_d, validate_d, model):    
    train_x = train_d.drop(columns=['WEEK_END_DATE', 'UNITS'])
    train_y = train_d['UNITS']
    
    valid_x = validate_d.drop(columns=['WEEK_END_DATE', 'UNITS'])
    valid_y = validate_d['UNITS']
    
    model.fit(train_x, train_y)
    
    predict_train = model.predict(train_x)
    predict_train = predict_train.clip(min=0)
            
    predict_validate = model.predict(valid_x)
    predict_validate = predict_validate.clip(min=0)
    
    return get_msle(train_y, predict_train), get_msle(valid_y, predict_validate), train_x
    
    
    
# function will extract the train and validation set using validation set dataframe
# The defined model will train on each of the set and the average RMSLE on train and validate set will be returned
def train_model(df, data, model):
    
    model_results_train = []
    model_results_valid = []
    for row in tqdm(range(df.shape[0]),leave=False, desc='training_model'):
        
        row = df.iloc[row]
        train_set = data[(data.WEEK_END_DATE >= row['train_start']) & (data.WEEK_END_DATE <= row['train_end'])]
        validate_set = data[data.WEEK_END_DATE == row['validate_week']]        
        train, valid, data_train = my_model(train_set,validate_set, model)
        model_results_train.append(train)
        model_results_valid.append(valid)
        
    return np.mean(model_results_train) , np.mean(model_results_valid), data_train

In [76]:
merged_data.dtypes




WEEK_END_DATE                  datetime64[ns]
STORE_NUM                               int64
UPC                                     int64
BASE_PRICE                            float64
FEATURE                                 int64
DISPLAY                                 int64
UNITS                                   int64
DESCRIPTION                            object
MANUFACTURER                           object
CATEGORY                               object
SUB_CATEGORY                           object
PRODUCT_SIZE                          float64
num_alternatives                        int64
num_lower_size_alternatives             int64
STORE_ID                                int64
STORE_NAME                             object
ADDRESS_CITY_NAME                      object
ADDRESS_STATE_PROV_CODE                object
MSA_CODE                                int64
SEG_VALUE_NAME                         object
PARKING_SPACE_QTY                     float64
SALES_AREA_SIZE_NUM               

In [77]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
# Step 1: Drop Unnecessary Columns

columns_to_drop = ['STORE_NUM', 'UPC', 'DESCRIPTION', 'CATEGORY', 'STORE_ID', 'STORE_NAME','ADDRESS_CITY_NAME', 'ADDRESS_STATE_PROV_CODE', 'MSA_CODE', 'year']
merged_data = merged_data.drop(columns=columns_to_drop)




In [84]:
# Step 2: One-Hot Encoding
merged_data = pd.get_dummies(merged_data, columns=['MANUFACTURER', 'SUB_CATEGORY', 'SEG_VALUE_NAME'])


KeyError: "None of [Index(['MANUFACTURER', 'SUB_CATEGORY', 'SEG_VALUE_NAME'], dtype='object')] are in the [columns]"

In [85]:
## merged_data -- drop --  'week_number'

## merged_data -- MANUFACTURER,SUB_CATEGORY,SEG_VALUE_NAME

## Convert to int BASE_PRICE,FEATURE,DISPLAY,UNITS,PRODUCT_SIZE,num_alternatives,num_lower_size_alternatives,PARKING_SPACE_QTY,
## SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS,'units_sold_prev_year'


## new columns units_sold_prev_year,PRODUCT_SIZE,PRODUCT_SIZE,PARKING_SPACE_QTY,num_alternatives,num_lower_size_alternatives,SEG_VALUE_NAME,units_sold_prev_year


# Step 3: Convert Specified Columns to Float
columns_to_convert = ['BASE_PRICE', 'UNITS', 'PRODUCT_SIZE', 'num_alternatives','num_lower_size_alternatives', 'PARKING_SPACE_QTY', 'SALES_AREA_SIZE_NUM','AVG_WEEKLY_BASKETS', 'units_sold_prev_year']
merged_data[columns_to_convert] = merged_data[columns_to_convert].astype(float)




In [86]:
# Step 4: Normalize the Float Columns
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()  # You can also use MinMaxScaler() if you prefer
merged_data[columns_to_convert] = scaler.fit_transform(merged_data[columns_to_convert])

merged_data.head()

,WEEK_END_DATE,BASE_PRICE,FEATURE,DISPLAY,UNITS,PRODUCT_SIZE,num_alternatives,num_lower_size_alternatives,PARKING_SPACE_QTY,SALES_AREA_SIZE_NUM,...,SUB_CATEGORY_ADULT CEREAL,SUB_CATEGORY_ALL FAMILY CEREAL,SUB_CATEGORY_KIDS CEREAL,SUB_CATEGORY_MOUTHWASH/RINSES AND SPRAYS,SUB_CATEGORY_MOUTHWASHES (ANTISEPTIC),SUB_CATEGORY_PIZZA/PREMIUM,SUB_CATEGORY_PRETZELS,SEG_VALUE_NAME_MAINSTREAM,SEG_VALUE_NAME_UPSCALE,SEG_VALUE_NAME_VALUE
0,2009-01-14,0.100996,0,0,0.007222,0.028056,1.0,0.2,0.097177,0.183985,...,False,False,False,False,False,False,True,False,False,True
1640,2009-01-21,0.100996,0,0,0.013333,0.028056,1.0,0.2,0.097177,0.183985,...,False,False,False,False,False,False,True,False,False,True
3276,2009-01-28,0.071124,0,0,0.003889,0.028056,1.0,0.2,0.097177,0.183985,...,False,False,False,False,False,False,True,False,False,True
4912,2009-02-04,0.073969,0,0,0.006667,0.028056,1.0,0.2,0.097177,0.183985,...,False,False,False,False,False,False,True,False,False,True
6552,2009-02-11,0.091038,0,0,0.008889,0.028056,1.0,0.2,0.097177,0.183985,...,False,False,False,False,False,False,True,False,False,True


In [87]:

merged_data.dtypes


WEEK_END_DATE                               datetime64[ns]
BASE_PRICE                                         float64
FEATURE                                              int64
DISPLAY                                              int64
UNITS                                              float64
PRODUCT_SIZE                                       float64
num_alternatives                                   float64
num_lower_size_alternatives                        float64
PARKING_SPACE_QTY                                  float64
SALES_AREA_SIZE_NUM                                float64
AVG_WEEKLY_BASKETS                                 float64
week_number                                         UInt32
units_sold_prev_year                               float64
MANUFACTURER_FRITO LAY                                bool
MANUFACTURER_GENERAL MI                               bool
MANUFACTURER_KELLOGG                                  bool
MANUFACTURER_P & G                                    bo

In [90]:



features_with_new_cols = merged_data

# Features without the new columns
features_without_new_cols = merged_data.drop(columns=['num_alternatives','num_lower_size_alternatives','week_number','units_sold_prev_year','PARKING_SPACE_QTY','week_number'])

# Split the data into training and testing sets for both versions

# Initialize the Random Forest Classifier
model_RFR = RandomForestRegressor(max_depth=15, n_estimators=75, n_jobs=-1)

valid_df = validation_df(features_with_new_cols, week, no_of_months=2, no_of_validation= 14)

rmsle_train, rmsle_valid, data_train = train_model(valid_df,features_with_new_cols, model_RFR)

print('RMSLE on train set: with the new columns ', rmsle_train)
print('RMSLE on validation set: with the new columns ', rmsle_valid)

training_model:   0%|          | 0/14 [00:00<?, ?it/s]

RMSLE on train set: with the new columns  0.0036021292004435805
RMSLE on validation set: with the new columns  0.013516333850542015


In [91]:
model_RFR = RandomForestRegressor(max_depth=15, n_estimators=75, n_jobs=-1)

valid_df = validation_df(features_without_new_cols, week, no_of_months=2, no_of_validation= 14)

rmsle_train, rmsle_valid, data_train = train_model(valid_df,features_without_new_cols, model_RFR)

print('RMSLE on train set: without the new columns  ', rmsle_train)
print('RMSLE on validation set: without the new columns ', rmsle_valid)

training_model:   0%|          | 0/14 [00:00<?, ?it/s]

RMSLE on train set: without the new columns   0.005620156787588419
RMSLE on validation set: without the new columns  0.0126781677821912
